In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [4]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# ChatPromptTemplate 사용

# 1단계 체인: 장르를 입력받아 영화 1편 추천
prompt1 = ChatPromptTemplate.from_template(
    [
        ("system", "당신은 영화 추천 전문가입니다."),
        ("human", "{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")
    ]
)

# 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
prompt2 = ChatPromptTemplate.from_template(
    "{movie} 추전한 영화의 제목을 먼저 알려주시고 추전한 영화의 줄거리를 3문장으로 요약해 주세요."
)



In [5]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000184B3F8C620> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000184B3F1A8D0> root_client=<openai.OpenAI object at 0x00000184B3F8EAE0> root_async_client=<openai.AsyncOpenAI object at 0x00000184B3F186E0> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [6]:
# 두 체인을 LCEL로 연결
chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

In [10]:
user_input = input("장르 입력(예: 액션, 로맨스, 공포 등): ")

In [11]:
response = chain2.invoke({"genre": user_input})
print("1단계 결과:", chain1.invoke({"genre": user_input}))
print("2단계 결과:", response)

1단계 결과: 공포 장르에서 추천할 만한 영화는 **<겟 아웃>(Get Out, 2017)**입니다.

2017년에 개봉한 <겟 아웃>은 조던 파일 감독의 데뷔작으로, 미국에서 큰 인기를 끌었습니다. 이 영화는 흑인 남성이 백인 여자 친구의 집에 초대되어 겪는 기이한 경험을 다루고 있으며, 하이브리드 호러 영화로 분류됩니다. 

영화는 공포와 스릴러를 결합하여 관객에게 긴장감과 두려움을 선사합니다. 또한, 이 영화는 사회적인 문제에 대한 은유적인 표현을 포함하고 있어 깊은 메시지를 전달합니다.

<겟 아웃>은 공포 영화 팬들에게 강력히 추천할 만한 작품입니다.
2단계 결과: 1954년 미국에 있는 한 정신병동에서 벌어지는 미스터리한 사건을 다루고 있습니다.

테디 다니엘스 경사는 아내가 죽은 사건으로 정신병동에 입원한 환자를 조사하게 됩니다.

그러나 테디는 진실을 추구하면서도 자신도 모르게 미스터리에 빠져듭니다.
